## Code for downloading and building CDC NEPH Dataset


In [12]:
// using county level data for race can make it so that i can use race as an interaction. can look at differences in sex across counties and how those ///
// differential outcomes change by racial demographics in the counties. AKA it's not showing that wildfires are shaping racial disparities for individuals ///
// rather, how wildfires are shaping asthma outcomes for diffferent counties based on their racial demographic.

In [74]:
// Setting up Data: Upload 
// CDC NEPHT Asthma Rates- Hospitalizations for Age-adjusted Rate of Hospitalizations for Asthma per ///
// 10,000 Population

cd "/Users/emilyernst/Desktop/Disasters-and-Health"
import delimited using "data/NEPH- Asthma/d_cdc_hospital_use_asthma.csv", clear

/Users/emilyernst/Desktop/Disasters-and-Health
(encoding automatically selected: ISO-8859-2)
(9 vars, 580 obs)


In [75]:
!curl 'https://ephtracking.cdc.gov/DataExplorer/getDataExport/103/10/1/0/false/:_:apigateway:_:api:_:v1:_:getCoreHolder:_:103:_:10:_:1:_:6:_:1:_:2020,2019,2018,2017,2016:_:1:_:0:-:GenderId=1,2?GenderId=1,2' ///
  -H 'Content-Type: application/json' ///
  --data-raw '{"geographicTypeIdFilter":"1","geographicItemsFilter":"6","temporalTypeIdFilter":null,"temporalItemsFilter":"2020,2019,2018,2017,2016","embedId":"","embedParentUrl":""}' ///
  > envirodata.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  376k  100  375k  100   168   233k    104  0:00:01  0:00:01 --:--:--  233k


In [76]:
unzipfile envirodata.zip

    inflating: data_011048.csv
    inflating: footnotes_011048.htm
    inflating: General_Information.pdf

successfully unzipped envirodata.zip to current directory
total processed:  3
        skipped:  0
      extracted:  3


In [77]:
//realized that every time i unzip the csv filename changes to a different name. is there a work around?
//using the .csv initially created
insheet using data/data_134614.csv, clear comma

(9 vars, 580 obs)


In [22]:
%head

,statefips,state,countyfips,county,year,value,datacomment,v8,gender
1,6,California,6001,Alameda,2016,4.3,.,.,Male
2,6,California,6001,Alameda,2016,4.8,.,.,Female
3,6,California,6001,Alameda,2017,4.4,.,.,Male
4,6,California,6001,Alameda,2017,4.9,.,.,Female
5,6,California,6001,Alameda,2018,3.6,.,.,Male


In [78]:
// Keeping only variables I need (value and datacomment are empty/full of missing observations)
keep countyfips county year value gender

In [79]:
//order pulls those specified variables to the front
//sort is going to list the observastions alphabetically/numerically
order countyfips year
sort  countyfips year

In [80]:
rename value hospital_10k

In [81]:
tab year


       Year |      Freq.     Percent        Cum.
------------+-----------------------------------
       2016 |        116       20.00       20.00
       2017 |        116       20.00       40.00
       2018 |        116       20.00       60.00
       2019 |        116       20.00       80.00
       2020 |        116       20.00      100.00
------------+-----------------------------------
      Total |        580      100.00


In [82]:
%head

,countyfips,year,county,hospital_10k,gender
1,6001,2016,Alameda,4.3,Male
2,6001,2016,Alameda,4.8,Female
3,6001,2017,Alameda,4.4,Male
4,6001,2017,Alameda,4.9,Female
5,6001,2018,Alameda,3.6,Male


In [83]:
//NOTE: 2 values per year per county is for male/female asthma rates.

In [84]:
//destring code for other datasets that need it:
*destring year, replace
*destring countyfips, replace

In [85]:
// Changing gender from string into categorical variable named "sex"
// So now instead of female and male it's numerical response categories (so we can see which number is associated with each label)
encode gender, gen(sex)

In [86]:
// Checking encode command

tab sex gender
tab sex, nol


           |        Gender
    Gender |    Female       Male |     Total
-----------+----------------------+----------
    Female |       290          0 |       290 
      Male |         0        290 |       290 
-----------+----------------------+----------
     Total |       290        290 |       580 

     Gender |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |        290       50.00       50.00
          2 |        290       50.00      100.00
------------+-----------------------------------
      Total |        580      100.00


In [87]:
// Dropping the string variable for gender

drop sex

In [88]:
// Reshaping from long to wide data format:
// Changing the format of how it looks if I were to pull it up in excel. This is done because otherwise it reads as a duplicate ///
// The original format had 2 observations for the same year and county by sex. Now there's only one observation for the county///
// for a certain year that has a female and a male rate for asthma hospitalizations. 

reshape wide hospital_10k, i(countyfips year) j(gender) string

(j = Female Male)

Data                               Long   ->   Wide
-----------------------------------------------------------------------------
Number of observations              580   ->   290         
Number of variables                   5   ->   5           
j variable (2 values)            gender   ->   (dropped)
xij variables:
                           hospital_10k   ->   hospital_10kFemale hospital_10kM
> ale
-----------------------------------------------------------------------------


In [89]:
%head

,countyfips,year,hospital_10kFemale,hospital_10kMale,county
1,6001,2016,4.8,4.3,Alameda
2,6001,2017,4.9,4.4,Alameda
3,6001,2018,4.2,3.6,Alameda
4,6001,2019,4.5,3.9,Alameda
5,6001,2020,2.2,1.6,Alameda


In [90]:
// Checking for duplicates

quietly by countyfips year: gen dup= cond(_N==1,0,_n)

In [91]:
ds 

countyfips    hospita~male  county
year          hospita~Male  dup


In [92]:
save data/asthma.dta, replace

file data/asthma.dta saved


In [39]:
//Setting up Data: Upload
// CDC ACS Census Data: Racial demographics of CA by county
import delimited using "data/CENSUS Data- Race/Census_ACS_Race.csv" , clear

(encoding automatically selected: ISO-8859-1)
(9 vars, 2,030 obs)


In [93]:
!curl 'https://ephtracking.cdc.gov/DataExplorer/getDataExport/645/13/0/0/false/:_:apigateway:_:api:_:v1:_:getCoreHolder:_:645:_:13:_:1:_:6:_:1:_:2020,2019,2018,2017,2016:_:0:_:0:-:RaceEthnicityId=2,3,4,5,6,7,8?RaceEthnicityId=2,3,4,5,6,7,8' ///
  -H 'Content-Type: application/json' ///
  --data-raw '{"geographicTypeIdFilter":"1","geographicItemsFilter":"6","temporalTypeIdFilter":null,"temporalItemsFilter":"2020,2019,2018,2017,2016","embedId":"","embedParentUrl":""}' ///
    > census.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  382k  100  382k  100   168   248k    109  0:00:01  0:00:01 --:--:--  248k


In [94]:
unzipfile census.zip

    inflating: data_011224.csv
    inflating: footnotes_011224.htm
    can't overwrite "General_Information.pdf"; skipping

successfully unzipped census.zip to current directory
total processed:  3
        skipped:  1
      extracted:  2


In [95]:
insheet using data/data_135343.csv, clear comma

(9 vars, 2,030 obs)


In [96]:
%head

,statefips,state,countyfips,county,year,value,datacomment,v8,raceethnicity
1,6,California,6001,Alameda,2016,50.5%,.,.,White including Hispanic
2,6,California,6001,Alameda,2016,11.5%,.,.,Black including Hispanic
3,6,California,6001,Alameda,2016,5.4%,.,.,Other including Hispanic
4,6,California,6001,Alameda,2016,31.5%,.,.,Asian/Pacific Islander including Hispanic
5,6,California,6001,Alameda,2016,1.1%,.,.,American Indian/Alaskan Native including Hispanic


In [97]:
ds

statefips     countyfips    year          datacomment   raceethnic~y
state         county        value         v8


In [98]:
keep countyfips year raceethnicity value

In [99]:
// Turning raceethnicity from string into a numeric variable "race"

encode raceethnicity, generate(race)

In [100]:
drop raceethnicity

In [101]:
tab race
tab race, nol


                         Race Ethnicity |      Freq.     Percent        Cum.
----------------------------------------+-----------------------------------
 All Non-White Races including Hispanic |        290       14.29       14.29
American Indian/Alaskan Native includin |        290       14.29       28.57
Asian/Pacific Islander including Hispan |        290       14.29       42.86
               Black including Hispanic |        290       14.29       57.14
                     Hispanic All Races |        290       14.29       71.43
               Other including Hispanic |        290       14.29       85.71
               White including Hispanic |        290       14.29      100.00
----------------------------------------+-----------------------------------
                                  Total |      2,030      100.00

       Race |
  Ethnicity |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |        290       14.29       14.29
  

In [102]:
// Reshaping format of value variable based on race

reshape wide value, i(countyfips year) j(race)

(j = 1 2 3 4 5 6 7)

Data                               Long   ->   Wide
-----------------------------------------------------------------------------
Number of observations            2,030   ->   290         
Number of variables                   4   ->   9           
j variable (7 values)              race   ->   (dropped)
xij variables:
                                  value   ->   value1 value2 ... value7
-----------------------------------------------------------------------------


In [103]:
%head

,countyfips,year,value1,value2,value3,value4,value5,value6,value7
1,6001,2016,49.5%,1.1%,31.5%,11.5%,22.3%,5.4%,50.5%
2,6001,2017,50.1%,1.1%,32.2%,11.3%,22.3%,5.4%,49.9%
3,6001,2018,50.6%,1.1%,33.0%,11.1%,22.2%,5.5%,49.4%
4,6001,2019,51.1%,1.1%,33.6%,10.9%,22.2%,5.5%,48.9%
5,6001,2020,51.5%,1.1%,34.0%,10.8%,22.4%,5.6%,48.5%


In [104]:
// Renaming all race categories

rename value2 native
label var native "American Indian/Alaskan Native including Hispanic"
rename value3 api
label var api "Asian/Pacific Islander including Hispanic"
rename value4 black
label var black "Black including Hispanic"
rename value5 hispanic
label var hispanic "Hispanic All Races"
rename value6 otherhispanic
label var otherhispanic"Other Race, Hispanic"
rename value7 white
label var white "white including Hispanic"
rename value1 nonwhite
label var nonwhite "All Non-White Races including Hispanic"

In [105]:
%head

,countyfips,year,nonwhite,native,api,black,hispanic,otherhispanic,white
1,6001,2016,49.5%,1.1%,31.5%,11.5%,22.3%,5.4%,50.5%
2,6001,2017,50.1%,1.1%,32.2%,11.3%,22.3%,5.4%,49.9%
3,6001,2018,50.6%,1.1%,33.0%,11.1%,22.2%,5.5%,49.4%
4,6001,2019,51.1%,1.1%,33.6%,10.9%,22.2%,5.5%,48.9%
5,6001,2020,51.5%,1.1%,34.0%,10.8%,22.4%,5.6%,48.5%


In [106]:
order countyfips year
sort countyfips year

In [107]:
// Turning all the value variables from string into numeric variable. You can tell it's string variable because of the ///
//percent sign (even though they're numbers, the % makes it a string)

foreach x in nonwhite-white {
destring `x', replace ignore("%")
}

nonwhite: character % removed; replaced as double
native: character % removed; replaced as double
api: character % removed; replaced as double
black: character % removed; replaced as double
hispanic: character % removed; replaced as double
otherhispanic: character % removed; replaced as double
white: character % removed; replaced as double


In [108]:
%head

,countyfips,year,nonwhite,native,api,black,hispanic,otherhispanic,white
1,6001,2016,49.5,1.1,31.5,11.5,22.3,5.4,50.5
2,6001,2017,50.1,1.1,32.2,11.3,22.3,5.4,49.9
3,6001,2018,50.6,1.1,33,11.1,22.2,5.5,49.4
4,6001,2019,51.1,1.1,33.6,10.9,22.2,5.5,48.9
5,6001,2020,51.5,1.1,34,10.8,22.4,5.6,48.5


In [109]:
// Checking for duplicates

quietly by countyfips year: gen dup= cond(_N==1,0,_n)

In [110]:
list if dup==1

In [111]:
drop dup

In [112]:
save data/race.dta, replace

file data/race.dta saved


In [114]:
//never do merge m:m!!!!!!
//can do 1:m m:1 but never m:m
merge 1:1 countyfips year using data/asthma.dta


    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                               290  (_merge==3)
    -----------------------------------------


In [115]:
drop _merge

In [116]:
save data/d_CDC_NEPH_asthma.dta, replace 

file data/d_CDC_NEPH_asthma.dta saved
